In [25]:
# from pyspark.mllib.linalg import Vectors
# from pyspark.mllib.regression import LabeledPoint
# from pyspark.mllib.evaluation import MulticlassMetrics

# preds = df.select(['label', 'features']) \
#                             .df.map(lambda line: (line[1], line[0]))
# metrics = MulticlassMetrics(preds)

# Confusion Matrix
# print(metrics.confusionMatrix().toArray())```

In [61]:
#import numpy
#import matplotlib.pyplot as plt

#x = numpy.random.uniform(0.0, 5.0, 250)

#plt.hist(x, 5)
#plt.show()

In [43]:
#vzorka_cela.crosstab("Accident_Severity", "Day_of_Week").show()

+-----------------------------+-----+-----+-----+-----+-----+-----+-----+
|Accident_Severity_Day_of_Week|    1|    2|    3|    4|    5|    6|    7|
+-----------------------------+-----+-----+-----+-----+-----+-----+-----+
|                            2|23942|29487|30915|30958|31556|34732|28784|
|                            1|  649|  542|  500|  495|  539|  741|  737|
+-----------------------------+-----+-----+-----+-----+-----+-----+-----+



In [ ]:
#vzorka_cela.saveAsTextFile('small_data.csv')

In [ ]:
#vzorka_cela.write.format('com.databricks.spark.csv').mode('append').option("header", "true").save('C:/Users/FIlip/Documents/TSVD/Zadanie')

In [1]:
import sys
from pyspark import SparkContext

sc = SparkContext(appName="example23")

print(u'Python version ' + sys.version)
print(u'Spark version ' + sc.version)

Python version 2.7.13 (v2.7.13:a06454b1afa1, Dec 17 2016, 20:53:40) [MSC v.1500 64 bit (AMD64)]
Spark version 2.2.0


In [2]:
from pyspark.sql import SparkSession
from pyspark.mllib.random import RandomRDDs
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import sys
from pyspark import SparkContext

spark = SparkSession.builder.appName("Zadanie").getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
# nacitanie
data=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Accidents.csv')
data1=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Vehicles.csv')
data2=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Casualties.csv')

In [4]:
data1=data1.withColumnRenamed("Accident_Index","ID")
data2=data2.withColumnRenamed("Accident_Index","IDE")

In [5]:
#spojenie dat a vymazanie potom tych duplikatych ID stlpcov
merge = data.join(data1, data.Accident_Index == data1.ID)
merge.drop("ID")
full_merge = merge.join(data2, merge.Accident_Index == data2.IDE)
full_merge=full_merge.drop("IDE")
full_merge=full_merge.drop("ID")


In [7]:
full_merge.head()

Row(Accident_Index=u'200501BS70192', Location_Easting_OSGR=526290, Location_Northing_OSGR=178770, Longitude=-0.182199, Latitude=51.493723, Police_Force=1, Accident_Severity=3, Number_of_Vehicles=2, Number_of_Casualties=1, Date=u'04/05/2005', Day_of_Week=4, Time=u'16:10', Local_Authority_(District)=12, Local_Authority_(Highway)=u'E09000020', 1st_Road_Class=5, 1st_Road_Number=0, Road_Type=6, Speed_limit=30, Junction_Detail=0, Junction_Control=-1, 2nd_Road_Class=-1, 2nd_Road_Number=0, Pedestrian_Crossing-Human_Control=0, Pedestrian_Crossing-Physical_Facilities=5, Light_Conditions=1, Weather_Conditions=1, Road_Surface_Conditions=1, Special_Conditions_at_Site=0, Carriageway_Hazards=0, Urban_or_Rural_Area=1, Did_Police_Officer_Attend_Scene_of_Accident=1, LSOA_of_Accident_Location=u'E01002835', Vehicle_Reference=1, Vehicle_Type=19, Towing_and_Articulation=0, Vehicle_Manoeuvre=18, Vehicle_Location-Restricted_Lane=0, Junction_Location=0, Skidding_and_Overturning=0, Hit_Object_in_Carriageway=0, 

In [6]:
# čiže ci cieľovy atribut je Accident_Severity a ma 3 hodnoty : 1, 2, 3 - jedna je smrtelna 2 je že važna ale nezomrel a 3 je že ľahka... tak som 3 replacol na 2 aby bolo len smrtelna
# a niesmrtelna
from pyspark.sql.functions import when
newsdf = full_merge.withColumn("Accident_Severity", when(full_merge["Accident_Severity"] == 3, 2).otherwise(full_merge["Accident_Severity"]))

In [7]:
newsdf.count()

4287593

In [8]:
#cize sampling  som si vytiahol koľko je počet mrtvych a koľko počet čo prežili nehodu a podľa toho som v tokom pomere zmenšil tie data na 5%
newsdf.registerTempTable("TempTable")
mrtvy = sqlContext.sql('SELECT * FROM TempTable WHERE Accident_Severity = 1')
mrtvy_pocet = mrtvy.count()
zivy = sqlContext.sql('SELECT * FROM TempTable WHERE Accident_Severity = 2')
zivy_pocet = zivy.count()

vzorka_mrtvy = mrtvy.sampleBy("Accident_Severity", fractions = {1: 1}, seed = 0)
vzorka_zivy = zivy.sampleBy("Accident_Severity", fractions = {2: 0.02}, seed = 0)

#tu ich nazad spojim a s tym datasetom treba pracovať ďalej
vzorka_cela = vzorka_mrtvy.union(vzorka_zivy)

In [10]:
vzorka_cela.select("Accident_Severity").show(20000)

+-----------------+
|Accident_Severity|
+-----------------+
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|
|                1|


In [11]:
print(vzorka_mrtvy.count())
print(vzorka_zivy.count())
print(vzorka_cela.count())

83607
83767
167374


In [9]:
#nahradenie neznamych hodnot atributov
atributes = ["Number_of_Vehicles",
			 "Number_of_Casualties",
			 "Junction_Detail",
			 "Junction_Control",
			 "Did_Police_Officer_Attend_Scene_of_Accident",
			 "Junction_Location",
			 "Skidding_and_Overturning",
			 "Hit_Object_off_Carriageway",
			 "Accident_Severity"]

vzorka_cela = vzorka_cela.select(atributes)

names2 = vzorka_cela.schema.names

for name in names2:
	type_counts = vzorka_cela.groupBy(name).count()
	type_counts = type_counts.orderBy(["count", name], ascending=[0, 1])
	moj_list = type_counts.select(name).collect()
	najcastejsia_hodnota = moj_list[0][0]
	vzorka_cela = vzorka_cela.withColumn(name, when(vzorka_cela[name] == -1,  najcastejsia_hodnota).otherwise(vzorka_cela[name]))

#kontrola ci nahradilo
for name in names2:
	x = vzorka_cela.filter(vzorka_cela[name] == -1).count()
	print(x)


0
0
0
79779
0
0
0
0
0


In [10]:
vzorka_cela = vzorka_cela.withColumn("Junction_Control", when(vzorka_cela["Junction_Control"] == -1, 2).otherwise(vzorka_cela["Junction_Control"]))


In [14]:
what= vzorka_cela.describe(["Number_of_Vehicles"])
what.collect()

[Row(summary=u'count', Number_of_Vehicles=u'167374'),
 Row(summary=u'mean', Number_of_Vehicles=u'2.9208359721342623'),
 Row(summary=u'stddev', Number_of_Vehicles=u'4.054891447468699'),
 Row(summary=u'min', Number_of_Vehicles=u'1'),
 Row(summary=u'max', Number_of_Vehicles=u'67')]

In [15]:
#rozdelenie na trenovaciu a testovaciu 
training_data, test_data = vzorka_cela.randomSplit([0.6, 0.4], seed=123)

In [16]:
vzorka_cela.count()

167374

In [17]:
print(vzorka_cela.take(1))
print("------")

[Row(Number_of_Vehicles=2, Number_of_Casualties=2, Junction_Detail=3, Junction_Control=4, Did_Police_Officer_Attend_Scene_of_Accident=1, Junction_Location=8, Skidding_and_Overturning=0, Hit_Object_off_Carriageway=0, Accident_Severity=1)]
------


In [18]:
print(training_data.take(1))
print("-----------")

[Row(Number_of_Vehicles=1, Number_of_Casualties=1, Junction_Detail=0, Junction_Control=0, Did_Police_Officer_Attend_Scene_of_Accident=1, Junction_Location=0, Skidding_and_Overturning=0, Hit_Object_off_Carriageway=4, Accident_Severity=1)]
-----------


In [19]:
print(test_data.take(1))

[Row(Number_of_Vehicles=1, Number_of_Casualties=1, Junction_Detail=0, Junction_Control=0, Did_Police_Officer_Attend_Scene_of_Accident=1, Junction_Location=0, Skidding_and_Overturning=1, Hit_Object_off_Carriageway=6, Accident_Severity=1)]


In [11]:
# #najprv si dropnem tie atributy ktore tam nechcem mat
vzorka_cela = vzorka_cela.drop(
					 "Local_Authority_(Highway)",
 					 "LSOA_of_Accident_Location",
					 "Time",
					 "Date",
					 "Accident_Index")


In [21]:
#do premennej names si ulozim mena atributov
names = vzorka_cela.schema.names
print(names)

#print(names.count)
#vytvorim si prazdny list correlations a do neho vo for cykle hodim korelacie kazdeho atributu s #atributom accident_severity, ale dajak to nefujguje zatial :D

correlations = []
for name in names:
    correlations.extend([vzorka_cela.stat.corr('Accident_Severity',name)])

['Number_of_Vehicles', 'Number_of_Casualties', 'Junction_Detail', 'Junction_Control', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Junction_Location', 'Skidding_and_Overturning', 'Hit_Object_off_Carriageway', 'Accident_Severity']


In [22]:
print(correlations)

[-0.14455292628599192, -0.21747366584706906, 0.19858182542061967, 0.21486248061423077, 0.2661458908345032, 0.20403999934927422, -0.14878846750244987, -0.15162677890272577, 1.0]


In [23]:
names = vzorka_cela.schema.names
t = zip(names,correlations)
#print(t)
#print("-------------------")
tt = spark.createDataFrame(t)
#tt.show()

tt.registerTempTable("TempTable")
atributy_table = sqlContext.sql('SELECT * FROM TempTable WHERE _2 > 0.05 OR _2 <-0.05')
atributy = atributy_table.select("_2")
atributy_table.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|  Number_of_Vehicles|-0.14455292628599192|
|Number_of_Casualties|-0.21747366584706906|
|     Junction_Detail| 0.19858182542061967|
|    Junction_Control| 0.21486248061423077|
|Did_Police_Office...|  0.2661458908345032|
|   Junction_Location| 0.20403999934927422|
|Skidding_and_Over...|-0.14878846750244987|
|Hit_Object_off_Ca...|-0.15162677890272577|
|   Accident_Severity|                 1.0|
+--------------------+--------------------+



In [24]:
vzorka_cela.count()

167374

In [25]:
Accident_Severity = vzorka_cela.describe(["Accident_Severity"])
Accident_Severity.collect()

[Row(summary=u'count', Accident_Severity=u'167374'),
 Row(summary=u'mean', Accident_Severity=u'1.5004779714890006'),
 Row(summary=u'stddev', Accident_Severity=u'0.5000012652101176'),
 Row(summary=u'min', Accident_Severity=u'1'),
 Row(summary=u'max', Accident_Severity=u'2')]

In [26]:
Number_of_Vehicles = vzorka_cela.describe(["Number_of_Vehicles"])
Number_of_Vehicles.collect()

[Row(summary=u'count', Number_of_Vehicles=u'167374'),
 Row(summary=u'mean', Number_of_Vehicles=u'2.9208359721342623'),
 Row(summary=u'stddev', Number_of_Vehicles=u'4.054891447468699'),
 Row(summary=u'min', Number_of_Vehicles=u'1'),
 Row(summary=u'max', Number_of_Vehicles=u'67')]

In [27]:
Number_of_Casualties = vzorka_cela.describe(["Number_of_Casualties"])
Number_of_Casualties.collect()

[Row(summary=u'count', Number_of_Casualties=u'167374'),
 Row(summary=u'mean', Number_of_Casualties=u'3.5452400014339145'),
 Row(summary=u'stddev', Number_of_Casualties=u'6.779365460029033'),
 Row(summary=u'min', Number_of_Casualties=u'1'),
 Row(summary=u'max', Number_of_Casualties=u'93')]

In [29]:
Road_Class = vzorka_cela.describe(["1st_Road_Class"])
Road_Class.collect()

AnalysisException: u"cannot resolve '`1st_Road_Class`' given input columns: [Number_of_Casualties, Junction_Location, Number_of_Vehicles, Accident_Severity, Junction_Control, Did_Police_Officer_Attend_Scene_of_Accident, Junction_Detail, Hit_Object_off_Carriageway, Skidding_and_Overturning];;\n'Aggregate [cast(count('1st_Road_Class) as string) AS 1st_Road_Class#3023, cast(avg('1st_Road_Class) as string) AS 1st_Road_Class#3025, cast(stddev_samp('1st_Road_Class) as string) AS 1st_Road_Class#3035, cast(min('1st_Road_Class) as string) AS 1st_Road_Class#3037, cast(max('1st_Road_Class) as string) AS 1st_Road_Class#3039]\n+- Project [Number_of_Vehicles#1786, Number_of_Casualties#1825, Junction_Detail#1864, CASE WHEN (Junction_Control#1903 = -1) THEN 2 ELSE Junction_Control#1903 END AS Junction_Control#2324, Did_Police_Officer_Attend_Scene_of_Accident#1942, Junction_Location#1980, Skidding_and_Overturning#2018, Hit_Object_off_Carriageway#2056, Accident_Severity#2093]\n   +- Project [Number_of_V

In [30]:
Speed_limit = vzorka_cela.describe(["Speed_limit"])
Speed_limit.collect()

AnalysisException: u"cannot resolve '`Speed_limit`' given input columns: [Number_of_Casualties, Junction_Location, Number_of_Vehicles, Accident_Severity, Junction_Control, Did_Police_Officer_Attend_Scene_of_Accident, Junction_Detail, Hit_Object_off_Carriageway, Skidding_and_Overturning];;\n'Aggregate [cast(count('Speed_limit) as string) AS Speed_limit#3051, cast(avg('Speed_limit) as string) AS Speed_limit#3053, cast(stddev_samp('Speed_limit) as string) AS Speed_limit#3063, cast(min('Speed_limit) as string) AS Speed_limit#3065, cast(max('Speed_limit) as string) AS Speed_limit#3067]\n+- Project [Number_of_Vehicles#1786, Number_of_Casualties#1825, Junction_Detail#1864, CASE WHEN (Junction_Control#1903 = -1) THEN 2 ELSE Junction_Control#1903 END AS Junction_Control#2324, Did_Police_Officer_Attend_Scene_of_Accident#1942, Junction_Location#1980, Skidding_and_Overturning#2018, Hit_Object_off_Carriageway#2056, Accident_Severity#2093]\n   +- Project [Number_of_Vehicles#1786, Number_of_Casualtie

In [31]:
Junction_Detail = vzorka_cela.describe(["Junction_Detail"])
Junction_Detail.collect()

[Row(summary=u'count', Junction_Detail=u'167374'),
 Row(summary=u'mean', Junction_Detail=u'1.8966207415727652'),
 Row(summary=u'stddev', Junction_Detail=u'2.54910867534019'),
 Row(summary=u'min', Junction_Detail=u'0'),
 Row(summary=u'max', Junction_Detail=u'9')]

In [32]:
Junction_Control = vzorka_cela.describe(["Junction_Control"])
Junction_Control.collect()

[Row(summary=u'count', Junction_Control=u'167374'),
 Row(summary=u'mean', Junction_Control=u'2.622976089476263'),
 Row(summary=u'stddev', Junction_Control=u'1.1913131472509995'),
 Row(summary=u'min', Junction_Control=u'0'),
 Row(summary=u'max', Junction_Control=u'4')]

In [34]:
nd_Road_Class = vzorka_cela.describe(["2nd_Road_Class"])
nd_Road_Class.collect()

AnalysisException: u"cannot resolve '`nd_Road_Class`' given input columns: [Number_of_Casualties, Junction_Location, Number_of_Vehicles, Accident_Severity, Junction_Control, Did_Police_Officer_Attend_Scene_of_Accident, Junction_Detail, Hit_Object_off_Carriageway, Skidding_and_Overturning];;\n'Aggregate [cast(count('nd_Road_Class) as string) AS nd_Road_Class#3271, cast(avg('nd_Road_Class) as string) AS nd_Road_Class#3273, cast(stddev_samp('nd_Road_Class) as string) AS nd_Road_Class#3283, cast(min('nd_Road_Class) as string) AS nd_Road_Class#3285, cast(max('nd_Road_Class) as string) AS nd_Road_Class#3287]\n+- Project [Number_of_Vehicles#1786, Number_of_Casualties#1825, Junction_Detail#1864, CASE WHEN (Junction_Control#1903 = -1) THEN 2 ELSE Junction_Control#1903 END AS Junction_Control#2324, Did_Police_Officer_Attend_Scene_of_Accident#1942, Junction_Location#1980, Skidding_and_Overturning#2018, Hit_Object_off_Carriageway#2056, Accident_Severity#2093]\n   +- Project [Number_of_Vehicles#178

In [35]:
Urban_or_Rural_Area = vzorka_cela.describe(["Urban_or_Rural_Area"])
Urban_or_Rural_Area.collect()

AnalysisException: u"cannot resolve '`Urban_or_Rural_Area`' given input columns: [Number_of_Casualties, Junction_Location, Number_of_Vehicles, Accident_Severity, Junction_Control, Did_Police_Officer_Attend_Scene_of_Accident, Junction_Detail, Hit_Object_off_Carriageway, Skidding_and_Overturning];;\n'Aggregate [cast(count('Urban_or_Rural_Area) as string) AS Urban_or_Rural_Area#3299, cast(avg('Urban_or_Rural_Area) as string) AS Urban_or_Rural_Area#3301, cast(stddev_samp('Urban_or_Rural_Area) as string) AS Urban_or_Rural_Area#3311, cast(min('Urban_or_Rural_Area) as string) AS Urban_or_Rural_Area#3313, cast(max('Urban_or_Rural_Area) as string) AS Urban_or_Rural_Area#3315]\n+- Project [Number_of_Vehicles#1786, Number_of_Casualties#1825, Junction_Detail#1864, CASE WHEN (Junction_Control#1903 = -1) THEN 2 ELSE Junction_Control#1903 END AS Junction_Control#2324, Did_Police_Officer_Attend_Scene_of_Accident#1942, Junction_Location#1980, Skidding_and_Overturning#2018, Hit_Object_off_Carriageway#20

In [41]:
Did_Police_Officer_Attend_Scene_of_Accident = vzorka_cela.describe(["Did_Police_Officer_Attend_Scene_of_Accident"])
Did_Police_Officer_Attend_Scene_of_Accident.collect()

[Row(summary=u'count', Did_Police_Officer_Attend_Scene_of_Accident=u'167374'),
 Row(summary=u'mean', Did_Police_Officer_Attend_Scene_of_Accident=u'1.0847622689306582'),
 Row(summary=u'stddev', Did_Police_Officer_Attend_Scene_of_Accident=u'0.2818549097359517'),
 Row(summary=u'min', Did_Police_Officer_Attend_Scene_of_Accident=u'1'),
 Row(summary=u'max', Did_Police_Officer_Attend_Scene_of_Accident=u'3')]

In [36]:
Vehicle_Manoeuvre = vzorka_cela.describe(["Vehicle_Manoeuvre"])
Vehicle_Manoeuvre.collect()

AnalysisException: u"cannot resolve '`Vehicle_Manoeuvre`' given input columns: [Number_of_Casualties, Junction_Location, Number_of_Vehicles, Accident_Severity, Junction_Control, Did_Police_Officer_Attend_Scene_of_Accident, Junction_Detail, Hit_Object_off_Carriageway, Skidding_and_Overturning];;\n'Aggregate [cast(count('Vehicle_Manoeuvre) as string) AS Vehicle_Manoeuvre#3327, cast(avg('Vehicle_Manoeuvre) as string) AS Vehicle_Manoeuvre#3329, cast(stddev_samp('Vehicle_Manoeuvre) as string) AS Vehicle_Manoeuvre#3339, cast(min('Vehicle_Manoeuvre) as string) AS Vehicle_Manoeuvre#3341, cast(max('Vehicle_Manoeuvre) as string) AS Vehicle_Manoeuvre#3343]\n+- Project [Number_of_Vehicles#1786, Number_of_Casualties#1825, Junction_Detail#1864, CASE WHEN (Junction_Control#1903 = -1) THEN 2 ELSE Junction_Control#1903 END AS Junction_Control#2324, Did_Police_Officer_Attend_Scene_of_Accident#1942, Junction_Location#1980, Skidding_and_Overturning#2018, Hit_Object_off_Carriageway#2056, Accident_Severity#

In [37]:
Junction_Location = vzorka_cela.describe(["Junction_Location"])
Junction_Location.collect()

[Row(summary=u'count', Junction_Location=u'167374'),
 Row(summary=u'mean', Junction_Location=u'1.8749686331210342'),
 Row(summary=u'stddev', Junction_Location=u'2.930789041721488'),
 Row(summary=u'min', Junction_Location=u'0'),
 Row(summary=u'max', Junction_Location=u'8')]

In [38]:
Skidding_and_Overturning = vzorka_cela.describe(["Skidding_and_Overturning"])
Skidding_and_Overturning.collect()

[Row(summary=u'count', Skidding_and_Overturning=u'167374'),
 Row(summary=u'mean', Skidding_and_Overturning=u'0.35559286388566924'),
 Row(summary=u'stddev', Skidding_and_Overturning=u'0.9137726358663378'),
 Row(summary=u'min', Skidding_and_Overturning=u'0'),
 Row(summary=u'max', Skidding_and_Overturning=u'5')]

In [39]:
Vehicle_Leaving_Carriageway = vzorka_cela.describe(["Vehicle_Leaving_Carriageway"])
Vehicle_Leaving_Carriageway.collect()

AnalysisException: u"cannot resolve '`Vehicle_Leaving_Carriageway`' given input columns: [Number_of_Casualties, Junction_Location, Number_of_Vehicles, Accident_Severity, Junction_Control, Did_Police_Officer_Attend_Scene_of_Accident, Junction_Detail, Hit_Object_off_Carriageway, Skidding_and_Overturning];;\n'Aggregate [cast(count('Vehicle_Leaving_Carriageway) as string) AS Vehicle_Leaving_Carriageway#3517, cast(avg('Vehicle_Leaving_Carriageway) as string) AS Vehicle_Leaving_Carriageway#3519, cast(stddev_samp('Vehicle_Leaving_Carriageway) as string) AS Vehicle_Leaving_Carriageway#3529, cast(min('Vehicle_Leaving_Carriageway) as string) AS Vehicle_Leaving_Carriageway#3531, cast(max('Vehicle_Leaving_Carriageway) as string) AS Vehicle_Leaving_Carriageway#3533]\n+- Project [Number_of_Vehicles#1786, Number_of_Casualties#1825, Junction_Detail#1864, CASE WHEN (Junction_Control#1903 = -1) THEN 2 ELSE Junction_Control#1903 END AS Junction_Control#2324, Did_Police_Officer_Attend_Scene_of_Accident#19

In [40]:
Hit_Object_off_Carriageway = vzorka_cela.describe(["Hit_Object_off_Carriageway"])
Hit_Object_off_Carriageway.collect()

[Row(summary=u'count', Hit_Object_off_Carriageway=u'167374'),
 Row(summary=u'mean', Hit_Object_off_Carriageway=u'0.9863539139890306'),
 Row(summary=u'stddev', Hit_Object_off_Carriageway=u'2.646979306562647'),
 Row(summary=u'min', Hit_Object_off_Carriageway=u'0'),
 Row(summary=u'max', Hit_Object_off_Carriageway=u'11')]

In [42]:
Casualty_Reference = vzorka_cela.describe(["Casualty_Reference"])
Casualty_Reference.collect()

AnalysisException: u"cannot resolve '`Casualty_Reference`' given input columns: [Number_of_Casualties, Junction_Location, Number_of_Vehicles, Accident_Severity, Junction_Control, Did_Police_Officer_Attend_Scene_of_Accident, Junction_Detail, Hit_Object_off_Carriageway, Skidding_and_Overturning];;\n'Aggregate [cast(count('Casualty_Reference) as string) AS Casualty_Reference#3708, cast(avg('Casualty_Reference) as string) AS Casualty_Reference#3710, cast(stddev_samp('Casualty_Reference) as string) AS Casualty_Reference#3720, cast(min('Casualty_Reference) as string) AS Casualty_Reference#3722, cast(max('Casualty_Reference) as string) AS Casualty_Reference#3724]\n+- Project [Number_of_Vehicles#1786, Number_of_Casualties#1825, Junction_Detail#1864, CASE WHEN (Junction_Control#1903 = -1) THEN 2 ELSE Junction_Control#1903 END AS Junction_Control#2324, Did_Police_Officer_Attend_Scene_of_Accident#1942, Junction_Location#1980, Skidding_and_Overturning#2018, Hit_Object_off_Carriageway#2056, Acciden

In [43]:
Casualty_Severity = vzorka_cela.describe(["Casualty_Severity"])
Casualty_Severity.collect()

AnalysisException: u"cannot resolve '`Casualty_Severity`' given input columns: [Number_of_Casualties, Junction_Location, Number_of_Vehicles, Accident_Severity, Junction_Control, Did_Police_Officer_Attend_Scene_of_Accident, Junction_Detail, Hit_Object_off_Carriageway, Skidding_and_Overturning];;\n'Aggregate [cast(count('Casualty_Severity) as string) AS Casualty_Severity#3736, cast(avg('Casualty_Severity) as string) AS Casualty_Severity#3738, cast(stddev_samp('Casualty_Severity) as string) AS Casualty_Severity#3748, cast(min('Casualty_Severity) as string) AS Casualty_Severity#3750, cast(max('Casualty_Severity) as string) AS Casualty_Severity#3752]\n+- Project [Number_of_Vehicles#1786, Number_of_Casualties#1825, Junction_Detail#1864, CASE WHEN (Junction_Control#1903 = -1) THEN 2 ELSE Junction_Control#1903 END AS Junction_Control#2324, Did_Police_Officer_Attend_Scene_of_Accident#1942, Junction_Location#1980, Skidding_and_Overturning#2018, Hit_Object_off_Carriageway#2056, Accident_Severity#

In [12]:
cast_vzorky= vzorka_cela.drop("Location_Easting_OSGR",
					 "Location_Northing_OSGR",
					"Longitude",
					 "Latitude",
					 "Local_Authority_(District)",
					 "1st_Road_Class",
					 "1st_Road_Number",
					 "Road_Type",
					 "Speed_limit",
					 "2nd_Road_Class",
					 "2nd_Road_Number",
					"Pedestrian_Crossing-Human_Control",
					"Pedestrian_Crossing-Physical_Facilities",
					 "Special_Conditions_at_Site",
					 "Carriageway_Hazards",
					 "Urban_or_Rural_Area",
					 "Vehicle_Reference",
					 "Towing_and_Articulation",
					 "Vehicle_Manoeuvre",
					"Vehicle_Location-Restricted_Lane",
					 "Vehicle_Leaving_Carriageway",
					 "Was_Vehicle_Left_Hand_Drive?",
					 "Journey_Purpose_of_Driver",
					 "Propulsion_Code",
					 "Driver_IMD_Decile",
					 "Driver_Home_Area_Type",
					"Vehicle_Reference",
					 "Casualty_Reference",
					 "Police_Force",
                     "Day_of_Week",
                     "Light_Conditions",
                     "Weather_Conditions",
                     "Road_Surface_Conditions",
                     "Vehicle_Type",
                     "Hit_Object_in_Carriageway",
                      "1st_Point_of_Impact",
                     "Sex_of_Driver",
                     "Age_of_Driver",
                    "Age_Band_of_Driver",
                    "Engine_Capacity_(CC)",
                    "Age_of_Vehicle",
                    "Casualty_Class",
                    "Sex_of_Casualty",
                    "Age_of_Casualty",
                    "Age_Band_of_Casualty",
                    "Casualty_Severity",
                    "Pedestrian_Location",
                    "Pedestrian_Movement",
                    "Car_Passenger",
                    "Bus_or_Coach_Passenger",
                    "Pedestrian_Road_Maintenance_Worker",
                    "Casualty_Type",
                    "Casualty_Home_Area_Type")
names = cast_vzorky.schema.names
print(names)

['Number_of_Vehicles', 'Number_of_Casualties', 'Junction_Detail', 'Junction_Control', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Junction_Location', 'Skidding_and_Overturning', 'Hit_Object_off_Carriageway', 'Accident_Severity']


In [45]:
training_data, test_data = cast_vzorky.randomSplit([0.6, 0.4], seed=123)

In [46]:
training_data.count()

100369

In [13]:
#Modelovanie
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.feature import VectorAssembler

vector_data = VectorAssembler(inputCols=["Number_of_Vehicles", "Number_of_Casualties", "Junction_Detail", "Junction_Control", "Did_Police_Officer_Attend_Scene_of_Accident", "Junction_Location","Skidding_and_Overturning","Hit_Object_off_Carriageway"],
        outputCol="features").transform(cast_vzorky)
training_data, test_data = vector_data.randomSplit([0.7, 0.3], seed=123)
 

In [48]:
vector_data.count()

167374

In [81]:
#Decision tree classifier
tree_classifier = DecisionTreeClassifier(featuresCol="features",labelCol="Accident_Severity",impurity="entropy",maxDepth=10, maxBins=100) 
 
tree_model = tree_classifier.fit(training_data)
 
predictions = tree_model.transform(test_data)

#print(tree_model.toDebugString)
test_error = predictions.filter(predictions["prediction"] != predictions["Accident_Severity"]).count() / float(test_data.count())
print "Testing error: {0:.4f}".format(test_error)


Testing error: 0.2791


In [82]:
# Select example rows to display.
predictions.select("prediction", "Accident_Severity", "features").show(5)

+----------+-----------------+--------------------+
|prediction|Accident_Severity|            features|
+----------+-----------------+--------------------+
|       2.0|                1|[1.0,1.0,0.0,0.0,...|
|       1.0|                1|[1.0,1.0,0.0,0.0,...|
|       2.0|                1|(8,[0,1,4],[1.0,1...|
|       1.0|                1|(8,[0,1,3,4],[1.0...|
|       1.0|                1|(8,[0,1,3,4],[1.0...|
+----------+-----------------+--------------------+
only showing top 5 rows



In [83]:
print(tree_model.toDebugString)


DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4a58bd090c3843f209c7) of depth 10 with 1069 nodes
  If (feature 1 <= 2.0)
   If (feature 4 <= 1.0)
    If (feature 5 <= 0.0)
     If (feature 1 <= 1.0)
      If (feature 7 <= 0.0)
       If (feature 0 <= 1.0)
        If (feature 6 <= 1.0)
         If (feature 3 <= 0.0)
          If (feature 6 <= 0.0)
           Predict: 1.0
          Else (feature 6 > 0.0)
           Predict: 1.0
         Else (feature 3 > 0.0)
          If (feature 6 <= 0.0)
           If (feature 2 <= 6.0)
            Predict: 1.0
           Else (feature 2 > 6.0)
            Predict: 1.0
          Else (feature 6 > 0.0)
           If (feature 2 <= 0.0)
            Predict: 1.0
           Else (feature 2 > 0.0)
            Predict: 1.0
        Else (feature 6 > 1.0)
         If (feature 3 <= 0.0)
          If (feature 6 <= 2.0)
           Predict: 2.0
          Else (feature 6 > 2.0)
           Predict: 1.0
         Else (feature 3 > 0.0)
          If (featu

In [84]:
y_true = predictions.select(['Accident_Severity']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

              precision    recall  f1-score   support

           1       0.70      0.76      0.73     24874
           2       0.74      0.68      0.71     25189

   micro avg       0.72      0.72      0.72     50063
   macro avg       0.72      0.72      0.72     50063
weighted avg       0.72      0.72      0.72     50063



array([[19009,  5865],
       [ 8109, 17080]], dtype=int64)

In [85]:
from sklearn.metrics import accuracy_score
print('Accuracy score: ', accuracy_score(y_true, y_pred))

('Accuracy score: ', 0.7208717016559135)


In [100]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'Accident_Severity', maxIter=10)
lrModel = lr.fit(training_data)
predictions = lrModel.transform(test_data)
predictions.select("prediction", "Accident_Severity", "features").show(10)
print(predictions)

+----------+-----------------+--------------------+
|prediction|Accident_Severity|            features|
+----------+-----------------+--------------------+
|       1.0|                1|[1.0,1.0,0.0,0.0,...|
|       1.0|                1|[1.0,1.0,0.0,0.0,...|
|       2.0|                1|(8,[0,1,4],[1.0,1...|
|       2.0|                1|(8,[0,1,3,4],[1.0...|
|       2.0|                1|(8,[0,1,3,4],[1.0...|
|       2.0|                1|(8,[0,1,3,4],[1.0...|
|       2.0|                1|(8,[0,1,3,4],[1.0...|
|       2.0|                1|(8,[0,1,3,4],[1.0...|
|       2.0|                1|[1.0,1.0,0.0,2.0,...|
|       1.0|                1|[1.0,1.0,0.0,2.0,...|
+----------+-----------------+--------------------+
only showing top 10 rows

DataFrame[Number_of_Vehicles: int, Number_of_Casualties: int, Junction_Detail: int, Junction_Control: int, Did_Police_Officer_Attend_Scene_of_Accident: int, Junction_Location: int, Skidding_and_Overturning: int, Hit_Object_off_Carriageway: int, A

In [101]:
y_true = predictions.select(['Accident_Severity']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

              precision    recall  f1-score   support

           1       0.71      0.66      0.68     24874
           2       0.68      0.74      0.71     25189

   micro avg       0.70      0.70      0.70     50063
   macro avg       0.70      0.70      0.70     50063
weighted avg       0.70      0.70      0.70     50063



array([[16296,  8578],
       [ 6646, 18543]], dtype=int64)

In [102]:
#presnost
from sklearn.metrics import accuracy_score
print('Accuracy score: ', accuracy_score(y_true, y_pred))

('Accuracy score: ', 0.69590316201586)


In [99]:
#ROC krivka
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol = 'Accident_Severity')
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 1.0


In [14]:
from pyspark.ml.feature import VectorAssembler

#SVM 
SVM_df = cast_vzorky.withColumn("Accident_Severity", when(cast_vzorky["Accident_Severity"] == 1, 0))
SVM_df1 = cast_vzorky.withColumn("Accident_Severity", when(cast_vzorky["Accident_Severity"] == 2, 1))

SVM_df=SVM_df.union(SVM_df1
                    
# no okey takže keďže to je binarny klasifikator tak tam možu byť len hodnoty 1/0 tak si musime accident severity z ½  upraviť (nova_vzorka maš ty nejako inak pomenovane):
#SVM_df = cast_vzorky.withColumn("Accident_Severity", when(cast_vzorky["Accident_Severity"] == 1, 0).otherwise(cast_vzorky["Accident_Severity"]))
#SVM_df.describe("Accident_Severity").collect()

#SVM_df = cast_vzorky.withColumn("Accident_Severity", when(cast_vzorky["Accident_Severity"] == 2, 1).otherwise(cast_vzorky["Accident_Severity"]))


#A potom samotny model v ktorom si špecifikuj nový vector data aj training a testing data (musia byť spravene z toho datasetu ktorý ma upravený accident severity na hodnoty 0 resp. 1



[Row(summary=u'count', Accident_Severity=u'167374'),
 Row(summary=u'mean', Accident_Severity=u'0.5004779714890006'),
 Row(summary=u'stddev', Accident_Severity=u'0.5000012652101175'),
 Row(summary=u'min', Accident_Severity=u'0'),
 Row(summary=u'max', Accident_Severity=u'1')]

In [23]:
from pyspark.ml.classification import LinearSVC
SVM_df = VectorAssembler(inputCols=["Number_of_Vehicles", "Number_of_Casualties", "Junction_Detail", 
                                             "Junction_Control", "Did_Police_Officer_Attend_Scene_of_Accident", 
                                             "Junction_Location","Skidding_and_Overturning","Hit_Object_off_Carriageway"],
        outputCol="features").transform(SVM_df) 

training_data, test_data = SVM_df.randomSplit([0.6, 0.4], seed=123)
training_data.collect()

svm_classifier = LinearSVC(featuresCol="features",labelCol="Accident_Severity")                  

svm_model = svm_classifier.fit(training_data)

predictions = svm_model.transform(test_data)

test_error = predictions.filter(predictions["prediction"] != predictions["Accident_Severity"]).count() / float(test_data.count())
print "Testing error: {0:.4f}".format(test_error)



Py4JJavaError: An error occurred while calling o1042.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 204 in stage 254.0 failed 1 times, most recent failure: Lost task 204.0 in stage 254.0 (TID 16854, localhost, executor driver): scala.MatchError: [null,1.0,(8,[0,1,3,4],[1.0,1.0,2.0,1.0])] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.classification.LinearSVC$$anonfun$5.apply(LinearSVC.scala:171)
	at org.apache.spark.ml.classification.LinearSVC$$anonfun$5.apply(LinearSVC.scala:171)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
	at scala.collection.TraversableOnce$class.foldLeft(TraversableOnce.scala:157)
	at scala.collection.AbstractIterator.foldLeft(Iterator.scala:1336)
	at scala.collection.TraversableOnce$class.aggregate(TraversableOnce.scala:214)


In [28]:
from pyspark.ml.classification import LinearSVC
SVM_df = VectorAssembler(inputCols=["Number_of_Vehicles", "Number_of_Casualties", "Junction_Detail", 
                                    "Junction_Control", "Did_Police_Officer_Attend_Scene_of_Accident", 
                                    "Junction_Location","Skidding_and_Overturning","Hit_Object_off_Carriageway"])
                .transform(SVM_df) 
        #outputCol="features").transform(SVM_df) 

SVMtraining_data, SVMtest_data = SVM_df.randomSplit([0.7, 0.3], seed=123)
#svm_classifier = LinearSVC(labelCol="Accident_Severity")                  

svm_model = svm_classifier.fit(SVMtraining_data)

predictions = svm_model.transform(SVMtest_data)

test_error = predictions.filter(predictions["prediction"] != predictions["Accident_Severity"]).count() / float(SVMtest_data.count())
print "Testing error: {0:.4f}".format(test_error)


Py4JJavaError: An error occurred while calling o1042.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 200 in stage 260.0 failed 1 times, most recent failure: Lost task 200.0 in stage 260.0 (TID 17091, localhost, executor driver): scala.MatchError: [null,1.0,(8,[0,1,4,6],[1.0,1.0,1.0,2.0])] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.classification.LinearSVC$$anonfun$5.apply(LinearSVC.scala:171)
	at org.apache.spark.ml.classification.LinearSVC$$anonfun$5.apply(LinearSVC.scala:171)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
	at scala.collection.TraversableOnce$class.foldLeft(TraversableOnce.scala:157)
	at scala.collection.AbstractIterator.foldLeft(Iterator.scala:1336)
	at scala.collection.TraversableOnce$class.aggregate(TraversableOnce.scala:214)


In [ ]:
#vyhodnotenie - SVM
predictions = svm_model.transform(SVMtest_data)

y_true = predictions.select(['Accident_Severity']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print classification_report(y_true, y_pred)
confusion_matrix(y_true, y_pred)

In [ ]:
#presnost SVM
from sklearn.metrics import accuracy_score
print('Accuracy score: ', accuracy_score(y_true, y_pred))

In [59]:
cast_vzorky

DataFrame[Number_of_Vehicles: int, Number_of_Casualties: int, Junction_Detail: int, Junction_Control: int, Did_Police_Officer_Attend_Scene_of_Accident: int, Junction_Location: int, Skidding_and_Overturning: int, Hit_Object_off_Carriageway: int, Accident_Severity: int]

In [60]:
# bayes 
from pyspark.ml .classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

vector_data = VectorAssembler(inputCols=["Number_of_Vehicles", "Number_of_Casualties", "Junction_Detail", "Junction_Control", "Did_Police_Officer_Attend_Scene_of_Accident", "Junction_Location","Skidding_and_Overturning","Hit_Object_off_Carriageway"],
        outputCol="features").transform(cast_vzorky)
training_data, test_data = vector_data.randomSplit([0.7, 0.3], seed=123)

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",featuresCol="features", labelCol="Accident_Severity")
 
# train the model
model = nb.fit(training_data)
 
predictions = model.transform(test_data)
#predictions.show()
 
evaluator = MulticlassClassificationEvaluator(labelCol="Accident_Severity", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))


Test set accuracy = 0.141142160877


In [61]:
print(model)

NaiveBayes_4157bd7a7f29a5e83de7


In [62]:
#vyhodnotenie - Bayes
predictions = model.transform(test_data)

y_true = predictions.select(['Accident_Severity']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print classification_report(y_true, y_pred)
confusion_matrix(y_true, y_pred)

c:\python27\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\python27\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       0.31      0.28      0.30     24874
         2.0       0.00      0.00      0.00     25189

   micro avg       0.14      0.14      0.14     50063
   macro avg       0.10      0.09      0.10     50063
weighted avg       0.15      0.14      0.15     50063



array([[    0,     0,     0],
       [17808,  7066,     0],
       [ 9521, 15668,     0]], dtype=int64)

In [63]:
#presnost Bayes
from sklearn.metrics import accuracy_score
print('Accuracy score: ', accuracy_score(y_true, y_pred))

('Accuracy score: ', 0.14114216087729461)


In [64]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="Accident_Severity", featuresCol="features",impurity="entropy", numTrees=10, maxBins=100)

# Train model.  This also runs the indexers.
model = rf.fit(training_data)

# Make predictions.
predictions = model.transform(test_data)

#print(model.toDebugString)`

test_error = predictions.filter(predictions["prediction"] != predictions["Accident_Severity"]).count() / float(test_data.count())
print "Testing error: {0:.4f}".format(test_error)

Testing error: 0.2899


In [65]:
print(model.toDebugString)

RandomForestClassificationModel (uid=RandomForestClassifier_4af8a05045c3fc256e85) with 10 trees
  Tree 0 (weight 1.0):
    If (feature 2 <= 0.0)
     If (feature 1 <= 2.0)
      If (feature 7 <= 0.0)
       If (feature 4 <= 1.0)
        If (feature 3 <= 0.0)
         Predict: 1.0
        Else (feature 3 > 0.0)
         Predict: 1.0
       Else (feature 4 > 1.0)
        If (feature 0 <= 1.0)
         Predict: 2.0
        Else (feature 0 > 1.0)
         Predict: 2.0
      Else (feature 7 > 0.0)
       If (feature 7 <= 5.0)
        If (feature 7 <= 3.0)
         Predict: 1.0
        Else (feature 7 > 3.0)
         Predict: 1.0
       Else (feature 7 > 5.0)
        If (feature 0 <= 1.0)
         Predict: 1.0
        Else (feature 0 > 1.0)
         Predict: 1.0
     Else (feature 1 > 2.0)
      If (feature 4 <= 1.0)
       If (feature 3 <= 0.0)
        If (feature 6 <= 0.0)
         Predict: 1.0
        Else (feature 6 > 0.0)
         Predict: 1.0
       Else (feature 3 > 0.0)
        If (f

In [66]:
#vyhodnotenie - RandomForest 
predictions = model.transform(test_data)

y_true = predictions.select(['Accident_Severity']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print classification_report(y_true, y_pred)
confusion_matrix(y_true, y_pred)

              precision    recall  f1-score   support

           1       0.71      0.70      0.71     24874
           2       0.71      0.72      0.71     25189

   micro avg       0.71      0.71      0.71     50063
   macro avg       0.71      0.71      0.71     50063
weighted avg       0.71      0.71      0.71     50063



array([[17425,  7449],
       [ 7063, 18126]], dtype=int64)

In [67]:
#presnost RandomForest 
from sklearn.metrics import accuracy_score
print('Accuracy score: ', accuracy_score(y_true, y_pred))

('Accuracy score: ', 0.7101252421948345)


In [68]:
predictions.select("prediction", "Accident_Severity", "features").show(5)

+----------+-----------------+--------------------+
|prediction|Accident_Severity|            features|
+----------+-----------------+--------------------+
|       1.0|                1|[1.0,1.0,0.0,0.0,...|
|       1.0|                1|[1.0,1.0,0.0,0.0,...|
|       2.0|                1|(8,[0,1,4],[1.0,1...|
|       2.0|                1|(8,[0,1,3,4],[1.0...|
|       2.0|                1|(8,[0,1,3,4],[1.0...|
+----------+-----------------+--------------------+
only showing top 5 rows



In [80]:
# Gradient boosted trees -vychadza Testing error: 1.0000
#Root Mean Squared Error (RMSE) on test data = 0.134178
#netušim či to je dobre … testing error 1 znamena že 100% chyba  ? :D 

from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier

#gbt_reg = GBTRegressor(featuresCol="features", labelCol="Accident_Severity", maxIter=10, maxBins=100)
 
# Train model. 
#model = gbt_reg.fit(training_data)
 
# Make predictions.
#predictions = model.transform(test_data)
 
# Select (prediction, true label) and compute test error
#evaluator = MulticlassClassificationEvaluator(
 #   labelCol="Accident_Severity", predictionCol="prediction", metricName="accuracy")
#accuracy = evaluator.evaluate(predictions)
#print("Test Error = %g" % (1.0 - accuracy))

gbt = GBTClassifier(labelCol="Accident_Severity", maxIter=10, maxBins=100)
gbtModel = gbt.fit(training_data)
predictions = gbtModel.transform(test_data)
predictions.select('Accident_Severity').show(10)


Py4JJavaError: An error occurred while calling o1892.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 201 in stage 1339.0 failed 1 times, most recent failure: Lost task 201.0 in stage 1339.0 (TID 117592, localhost, executor driver): java.lang.IllegalArgumentException: requirement failed: GBTClassifier was given dataset with invalid label 2.0.  Labels must be in {0,1}; note that GBTClassifier currently only supports binary classification.
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.classification.GBTClassifier$$anonfun$1.apply(GBTClassifier.scala:153)
	at org.apache.spark.ml.classification.GBTClassifier$$anonfun$1.apply(GBTClassifier.scala:151)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:216)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.a

In [70]:
print(predictions)

#vyhodnotenie - Gradient boosted trees
#predictions = model.transform(test_data)

#y_true = predictions.select(['Accident_Severity']).collect()
#y_pred = predictions.select(['prediction']).collect()

#from sklearn.metrics import classification_report, confusion_matrix
#print classification_report(y_true, y_pred)
#confusion_matrix(y_true, y_pred)

In [71]:
print(gbtModel)
#from sklearn.metrics import accuracy_score
#print('Accuracy score: ', accuracy_score(y_true, y_pred))

In [72]:
from pyspark.ml.clustering import KMeans
#Trains a k-means model.
kmeans = KMeans().setK(3).setSeed(1234)
model = kmeans.fit(training_data)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(training_data)
print("Within Set Sum of Squared Errors = " + str(wssse))
print("------------------------------------------------")
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)    
    
print("------------------------------------------------")
#detekovanie anomalii
for center in centers:
    for point in center:
        if (point > 5 or -5 > point):
               print "anomalia: {0:.15f}".format(point)

Within Set Sum of Squared Errors = 3494178.06765
------------------------------------------------
Cluster Centers: 
[2.69613341 2.93208365 0.68974215 2.23695423 1.07549526 0.32862355
 0.41082842 1.1557656 ]
[23.57383357 49.96151996  0.15873016  1.81818182  1.000481    0.17364117
  0.32227032  0.86964887]
[2.18821145 2.16973598 4.92809464 3.6056082  1.11276    5.75287562
 0.2217057  0.58227546]
------------------------------------------------
anomalia: 23.573833573833575
anomalia: 49.961519961519961
anomalia: 5.752875618332439
